In [1]:
import pandas as pd
import os

%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.insert(0, "..")
import os

os.chdir("..")

In [4]:
import pickle

In [5]:
with open("data/ground_truth/eval-run-v2-2025-11-30-11-15.bin", "rb") as f_in:
    rows = pickle.load(f_in)

/Users/yenchunchen/Desktop/Project/health-agent/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
len(rows)

4

In [11]:
import pandas as pd

pd.DataFrame(rows)

,question,answer,product_answer,product_reason_answer,messages,original_question,original_result
0,不該買的 sephora 粉底液,* Product: Isamaya妝前乳 ** Buy: False ** Reaso...,"[Isamaya妝前乳, Dior小奶瓶粉底]","[包裝設計使用不便，容易使產品堆積造成髒亂，並且在底妝上容易起屑。, 雖然它有多種色號並且提...","[{'kind': 'user-prompt', 'content': '不該買的 seph...","{'user_question': '不該買的 sephora 粉底液', 'title':...",AgentRunResult(output=YoutubeSummaryOutput(tit...
1,必買的 Sephora 粉底液,* Product: Dior 小奶瓶粉底 ** Buy: True ** Reason...,"[Dior 小奶瓶粉底, Patrick Ta 的蜜粉]","[非常不挑膚況，妝感自然，不厚粉。, 質地輕薄且有自然的柔焦效果。]","[{'kind': 'user-prompt', 'content': '必買的 Sepho...","{'user_question': '必買的 Sephora 粉底液', 'title': ...",AgentRunResult(output=YoutubeSummaryOutput(tit...
2,What are the must-buy foundations at Sephora?,* Product: Natasha Denona Hy-Glam Foundation ...,[Natasha Denona Hy-Glam Foundation],[它的質地輕薄，自然且光澤感很好，適合習慣使用粉底液的人。],"[{'kind': 'user-prompt', 'content': 'What are ...",{'user_question': 'What are the must-buy found...,AgentRunResult(output=YoutubeSummaryOutput(tit...
3,What are the don't-buy foundations at Sephora?,* Product: Milk Hydro Grip Foundation ** Buy:...,"[Milk Hydro Grip Foundation, Tower 28 Bouncing...",[The texture was initially disliked due to its...,"[{'kind': 'user-prompt', 'content': 'What are ...",{'user_question': 'What are the don't-buy foun...,AgentRunResult(output=YoutubeSummaryOutput(tit...


In [52]:
rows[0]["original_question"]["product"]

'[HAUS LABS Triclone Skin Tech Foundation #070 Fair Neutral, CHARLOTTE TILBURY Airbrush Flawless Foundation #2 Cool]'

In [7]:
from pydantic_ai import Agent
from pydantic import BaseModel

from pydantic import BaseModel, Field
from typing import List

from enum import Enum
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
openai_client = OpenAI()

In [111]:
from enum import Enum
from pydantic import BaseModel, Field

# class CheckName(str, Enum):
#     instructions_follow = "instructions_follow"
#     instructions_avoid = "instructions_avoid"
#     answer_relevant = "answer_relevant"
#     answer_clear = "answer_clear"

#     completeness = "completeness"
#     reason_relevant = "reason_relevant"
#     reason_citations = "reason_citations"
#     reason_completenes = "reason_completenes"
#     # answer_citations = "answer_citations"
#     # tool_call_search = "tool_call_search"

# CHECK_DESCRIPTIONS = {
#     CheckName.instructions_follow: "The agent followed the user's instructions (in <INSTRUCTIONS>)",
#     CheckName.instructions_avoid: "The agent avoided doing things it was told not to do",
#     CheckName.answer_relevant: "The response directly addresses the user's question",
#     CheckName.answer_clear: "The answer is clear and correct",
#     CheckName.completeness: "The response is complete and covers all key aspects of the request",
#     CheckName.reason_relevant: "The reasoning in (<DETAILS ANSWER>) is consistent with grounded in (<REFERENCE>)",
#     CheckName.reason_citations: "The reasoning correctly references the relevant products, items, or sources that appear in (<REFERENCE>)",
#     CheckName.reason_completenes: "The response in (<DETAILS ANSWER>) is complete and covers all key products grounded in (<REFERENCE>)",
#     # CheckName.tool_call_search: "Is the search tool invoked?"
# }


class CheckName(str, Enum):
    instructions_follow = "instructions_follow"
    instructions_avoid = "instructions_avoid"

    answer_relevant = "answer_relevant"
    answer_clear = "answer_clear"
    completeness = "completeness"

    reason_relevant = "reason_relevant"
    reason_reference_relevant = "reason_reference_relevant"
    reason_citations = "reason_citations"
    reason_completeness = "reason_completeness"


CHECK_DESCRIPTIONS = {
    CheckName.instructions_follow: "The agent followed all required instructions in <INSTRUCTIONS>.",
    CheckName.instructions_avoid: "The agent avoided doing anything that was explicitly forbidden.",
    # --- Answer-level evaluation ---
    CheckName.answer_relevant: "The final answer (<ANSWER>) directly addresses the user's question (<QUESTION>) "
    "without drifting into unrelated content.",
    CheckName.answer_clear: "The final answer (<ANSWER>) is clear, logically structured, and correct.",
    CheckName.completeness: "The final answer (<ANSWER>) fully covers all required aspects of the user request.",
    # --- Reasoning / reference-grounding evaluation ---
    CheckName.reason_relevant: "The reasoning in (<DETAILS ANSWER>) logically supports the final answer. "
    "It is relevant, non-contradictory, and stays within the scope of the question.",
    CheckName.reason_reference_relevant: "The reasoning in (<DETAILS ANSWER>) is consistent with the ground-truth (<REFERENCE>). "
    "It mentions the same key products/topics and uses similar reasoning logic. "
    "No hallucinated products or claims appear.",
    CheckName.reason_citations: "The reasoning correctly references the relevant products, items, or transcript sources "
    "that appear in (<REFERENCE>). It does not introduce content not present in <REFERENCE>.",
    CheckName.reason_completeness: "The reasoning in (<DETAILS ANSWER>) covers all products, topics, and key information "
    "present in the ground-truth (<REFERENCE>), without missing important details.",
}


class EvaluationCheck(BaseModel):
    check_name: CheckName = Field(description="The type of evaluation check")
    reasoning: str = Field(description="The reasoning behind the check result")
    check_pass: bool = Field(
        description="Whether the check passed (True) or failed (False)"
    )


class EvaluationChecklist(BaseModel):
    checklist: list[EvaluationCheck] = Field(
        description="List of all evaluation checks"
    )
    summary: str = Field(description="Evaluation summary")

In [112]:
def generate_checklist_text():
    checklist_items = []
    for check_name in CheckName:
        description = CHECK_DESCRIPTIONS[check_name]
        checklist_items.append(f"- {check_name.value}: {description}")
    return "\n".join(checklist_items)

In [113]:
eval_instructions = f"""
You are evaluating the quality of an AI agent’s responses: (<ANSWER>) and (<DETAILS ANSWER>)
to a user question (<QUESTION>). You are also given:

- (<LOG>): the full agent interaction log  
- (<REFERENCE>): the ground-truth detailed answer that the user question was originally derived from

Your task:
Evaluate whether (<DETAILS ANSWER>) is correct and SIMILAR based on (<REFERENCE>),
and evaluate whether (<ANSWER>) correctly answers (<QUESTION>).

Use the following checklist:

{generate_checklist_text()}

Output true/false for each check and provide a short explanation for your judgment.
"""

In [114]:
judge = Agent(
    name="judge",
    instructions=eval_instructions,
    model="gpt-4o-mini",
    output_type=EvaluationChecklist,
)

In [115]:
from service.agent.src.agent import youtube_agent

In [116]:
rows[0]["original_question"]

{'user_question': '不該買的 sephora 粉底液',
 'title': 'Sephora Summary: What to Buy and What Not to!',
 'product': '[HAUS LABS Triclone Skin Tech Foundation #070 Fair Neutral, CHARLOTTE TILBURY Airbrush Flawless Foundation #2 Cool]',
 'recommend': False,
 'reason': '我覺得Haus Labs的這款粉底液啊 它會有一點點黏黏的感覺 所以我覺得上在我的臉上 就第一個是上起來的 感受 就沒有這麼好 尤其是我拿粉撲在拍打的時候 我就會覺得粉底有一點推不開 它上臉的妝感沒有到特別的細緻 都還沒有完全拍開 我就已經想要把它卸掉了 它是一個非常非常高遮瑕又霧面 然後粉體妝感都很厚的一款粉底 不過這一款粉底它本來的定位 就比較偏像是紅毯'}

In [117]:
import json

json.dumps(rows[0]["messages"])

'[{"kind": "user-prompt", "content": "\\u4e0d\\u8a72\\u8cb7\\u7684 sephora \\u7c89\\u5e95\\u6db2"}, {"kind": "tool-call", "tool_name": "search_es", "args": {"query_text": "\\u4e0d\\u8a72\\u8cb7\\u7684 sephora \\u7c89\\u5e95\\u6db2"}}, {"kind": "tool-return", "content": [{"start_time": "1:57", "video_id": "2SvN45DKWFg", "content": "\\u5c31\\u6c92\\u6709\\u50cf\\u662f \\u6211\\u5176\\u4ed6\\u7684\\u599d\\u524d\\u4e73\\u6db2\\u4f86\\u5f97\\u9019\\u9ebc\\u5920 \\u6211\\u81ea\\u5df1\\u9084\\u662f\\u6bd4\\u8f03\\u559c\\u6b61\\u4ed6\\u5011 \\u9019\\u500b\\u6efe\\u6efe\\u4e73\\u7684\\u8cea\\u5730 \\u518d\\u4f86\\u9019\\u500bIsamaya\\u7684\\u599d\\u524d \\u4e5f\\u662f\\u6211\\u6700\\u8fd1\\u7528\\u5230\\u7684 \\u6211\\u7684\\u7b2c\\u4e00\\u5370\\u8c61\\u5176\\u5be6\\u9084\\u4e0d\\u932f \\u4f46\\u662f\\u5462\\u6211\\u5f8c\\u4f86\\u89ba\\u5f97 \\u5b83\\u7684\\u9019\\u500b\\u5305\\u88dd\\u5be6\\u5728\\u662f\\u592a\\u96e3\\u7528\\u4e86 \\u4f60\\u7528\\u4e00\\u7528\\u554a \\u5b83\\u7684\\u9019\\u500

In [118]:
rows[0]

{'question': '不該買的 sephora 粉底液',
 'answer': '* Product: Isamaya妝前乳  ** Buy: False  ** Reason: 包裝設計使用不便，容易使產品堆積造成髒亂，並且在底妝上容易起屑。  ** Transcript: 這個Isamaya的妝前，第一印象其實還不錯，但是呢我後來覺得它的這個包裝實在是太難用了....如果我擦了比較多，一沒有控制好用量的話，就比較容易跟我後續的底妝起屑打架。\n  [Video Reference]  Youtuber: 未知  Title: Sephora Summary: What to Buy and What Not to!  URL: https://www.youtube.com/watch?v=2SvN45DKWFg?t=117\n======================* Product: Dior小奶瓶粉底  ** Buy: False  ** Reason: 雖然它有多種色號並且提供光澤妝感，但使用體驗不如預期，特別是在遮瑕度上表現不佳。  ** Transcript: Dior除了剛剛的這個小奶瓶粉底之外，因為我真的太喜歡他們家的這個包裝，所以我又再去買了他們的這個Forever的黑蓋，這兩個都是光澤妝感，但是黑蓋的這個會再更有遮瑕度一點。\n  [Video Reference]  Youtuber: 未知  Title: Sephora Summary: What to Buy and What Not to!  URL: https://www.youtube.com/watch?v=2SvN45DKWFg?t=274\n======================',
 'product_answer': ['Isamaya妝前乳', 'Dior小奶瓶粉底'],
 'product_reason_answer': ['包裝設計使用不便，容易使產品堆積造成髒亂，並且在底妝上容易起屑。',
  '雖然它有多種色號並且提供光澤妝感，但使用體驗不如預期，特別是在遮瑕度上表現不佳。'],
 'messages': [{'kind': 'user-prompt', 'content': '不該買的 sephora 粉底液'},
  {'kind': 

In [119]:
import asyncio
from tqdm.auto import tqdm


async def map_progress(seq, f, max_concurrency=6):
    """Asynchronously map async function f over seq with progress bar."""
    semaphore = asyncio.Semaphore(max_concurrency)

    async def run(el):
        async with semaphore:
            return await f(el)

    # create one coroutine per element
    coros = [run(el) for el in seq]

    # turn them into tasks that complete as they finish
    completed = asyncio.as_completed(coros)

    results = []

    for coro in tqdm(completed, total=len(seq)):
        result = await coro
        results.append(result)

    return results

In [120]:
import json


async def run_eval(row):
    original_product = row["original_question"]["product"]
    original_recommend = row["original_question"]["recommend"]
    original_reasone = row["original_question"]["reason"]

    user_prompt = f"""
    <INSTRUCTIONS>{youtube_agent.beauty_instructions}</INSTRUCTIONS>
    <QUESTION>{row["question"]}</QUESTION>
    <FINAL ANSWER>{row["answer"]}</FINAL ANSWER>
    <DETAILS ANSWER>{row["product_answer"]}</DETAILS ANSWER>
    <REFERENCE>{original_product}</REFERENCE>
    <LOG>{json.dumps(row["messages"])}</LOG>
    """.strip()

    output = await judge.run(user_prompt=user_prompt)

    return row, output

In [121]:
results = await map_progress(rows, run_eval, max_concurrency=10)

  0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:08<00:00,  2.22s/it]


In [122]:
results[0][1]

AgentRunResult(output=EvaluationChecklist(checklist=[EvaluationCheck(check_name=<CheckName.instructions_follow: 'instructions_follow'>, reasoning="The agent followed all required instructions, addressing the user's question in Chinese and providing relevant product summaries.", check_pass=True), EvaluationCheck(check_name=<CheckName.instructions_avoid: 'instructions_avoid'>, reasoning='The agent avoided doing anything that was forbidden, such as providing unrelated content.', check_pass=True), EvaluationCheck(check_name=<CheckName.answer_relevant: 'answer_relevant'>, reasoning="The final answer discusses products related to the user's query.", check_pass=True), EvaluationCheck(check_name=<CheckName.answer_clear: 'answer_clear'>, reasoning='The answer is clear and structured, providing specific products and reasons for not purchasing them.', check_pass=True), EvaluationCheck(check_name=<CheckName.completeness: 'completeness'>, reasoning='The final answer covers the products and specific

In [123]:
all_checks = []

for original_row, result in results:
    checks = result.output.checklist
    checks_formatted = {"question": original_row["question"]}
    for check in checks:
        checks_formatted[check.check_name] = check.check_pass
        # checks_formatted[check.reasoning] = check.reasoning
    all_checks.append(checks_formatted)

In [124]:
all_checks[1]

{'question': "What are the don't-buy foundations at Sephora?",
 <CheckName.instructions_follow: 'instructions_follow'>: True,
 <CheckName.instructions_avoid: 'instructions_avoid'>: True,
 <CheckName.answer_relevant: 'answer_relevant'>: True,
 <CheckName.answer_clear: 'answer_clear'>: True,
 <CheckName.completeness: 'completeness'>: True,
 <CheckName.reason_relevant: 'reason_relevant'>: True,
 <CheckName.reason_reference_relevant: 'reason_reference_relevant'>: False,
 <CheckName.reason_citations: 'reason_citations'>: True,
 <CheckName.reason_completeness: 'reason_completeness'>: True}

In [125]:
df_eval = pd.DataFrame(all_checks)

In [126]:
df_eval

,question,CheckName.instructions_follow,CheckName.instructions_avoid,CheckName.answer_relevant,CheckName.answer_clear,CheckName.completeness,CheckName.reason_relevant,CheckName.reason_reference_relevant,CheckName.reason_citations,CheckName.reason_completeness
0,不該買的 sephora 粉底液,True,True,True,True,True,True,True,True,True
1,What are the don't-buy foundations at Sephora?,True,True,True,True,True,True,False,True,True
2,What are the must-buy foundations at Sephora?,True,True,True,True,True,True,True,True,True
3,必買的 Sephora 粉底液,True,True,True,True,True,True,False,False,False


In [127]:
df_eval[df_eval.columns[1:]].mean()

CheckName.instructions_follow          1.00
CheckName.instructions_avoid           1.00
CheckName.answer_relevant              1.00
CheckName.answer_clear                 1.00
CheckName.completeness                 1.00
CheckName.reason_relevant              1.00
CheckName.reason_reference_relevant    0.50
CheckName.reason_citations             0.75
CheckName.reason_completeness          0.75
dtype: float64